# Practical session 1: First steps with the simulator

In the `quickstart_tutorial`, we saw how to control the simulator using a Jupyter Notebook. Now, we will see how to make the agents move and implement basic behaviors.

First of all, we instantiate a `controller` (this will be done at the start of every notebook from now on)

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
start_server_and_interface(scene_name="session_1")

Interface process not found
Server process not found
Server and Interface Stopped
STARTING SERVER


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[2024-10-04 18:47:06,567][__main__][INFO] - Scene running: session_1
[2024-10-04 18:47:08,690][vivarium.simulator.simulator][INFO] - Simulator initialized
[2024-10-04 18:47:08,690][__main__][INFO] - Simulator server started

STARTING INTERFACE


2024-10-04 18:47:09,978 Starting Bokeh server version 3.3.4 (running on Tornado 6.4)
2024-10-04 18:47:09,978 User authentication hooks NOT provided (default user enabled)
2024-10-04 18:47:09,980 Bokeh app running at: http://localhost:5006/run_interface
2024-10-04 18:47:09,980 Starting Bokeh server with process id: 29994
2024-10-04 18:47:12,352 An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


self.hide_all_non_existing = <bound method EntityManager.hide_all_non_existing of <vivarium.interface.panel_app.AgentManager object at 0x7b22a4debc70>>
self.hide_all_non_existing = <bound method EntityManager.hide_all_non_existing of <vivarium.interface.panel_app.ObjectManager object at 0x7b229af26e60>>


2024-10-04 18:47:14,058 WebSocket connection opened
2024-10-04 18:47:14,069 ServerConnection created


In [3]:
controller = NotebookController()


[2024-10-04 18:47:17,417][vivarium.simulator.simulator][INFO] - 
Update attribute: attr = 'freq'; type_ = <class 'float'>


/home/cleger/Desktop/code/vivarium/venv/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


This time, only one agent should be present. We can check this with this instruction:

In [4]:
controller.agents

Next, we start the simulation:

In [5]:
controller.run(threaded=True)

Nothing should move for now, this is normal as the agents have all their motors stopped.

It can be the case that at one point during the session, the connection is lost and consequently you can't control the robot anymore. If you experience this problem, follow the steps below.

First, stop the simulator:

In [6]:
controller.stop()

Then restart this notebook by selecting Kernel -> Restart in the menu bar, and re-open the session by executing:

In [7]:
from vivarium.controllers.notebook_controller import NotebookController
controller = NotebookController()

Then restart the simulation

In [8]:
controller.run()

### Making the agents move

You should now know how to set the attributes of agents (see [the quickstart tutorial](https://github.com/clement-moulin-frier/vivarium/blob/main/notebooks/quickstart_tutorial.ipynb)). For this time, as there is only one agent, we can create a shortcut for the next cells of code:

In [9]:
agent = controller.agents[0]

In [10]:
agent.diameter = 10.

In [11]:
agent.color = 'green'

In [13]:
agent.infos()

Entity Overview:
--------------------
Entity Type: AGENT
Entity Idx: 0
Position: x=11.15, y=34.80

Sensors: Left=0.27, Right=0.00
Motors: Left=0.00, Right=0.00
Behaviors: None
Can eat: False
Diet: []
Eating range: 10



Now, the variable `agent` refers to the first (and only) agent of the simulation. You can simply write `agent` instead of `controller.agents[0]` for the next instructions.

**Q1:** What is the condition for the agent to turn left? to turn right? to move straight forward? to stop?

*This is a cell where you can write text instead of code.* Double click on this text and enter your answer here. Once it is done, press `Shift-Enter`.

**Q2:** How can you make the agent move forward without any rotation? Write the corresponding code in the cell below. 

In [14]:
agent.left_motor = agent.right_motor = 0.5

**Q3:** Now write the code making the agent stop:

In [15]:
agent.left_motor = agent.right_motor = 0.0

**Q4:** And the code to make it move in a large circle:

In [16]:
agent.left_motor = 0.95
agent.right_motor = 1.

Let's try more complex choregraphies. A useful function for this is the ability to wait for a given time by using `controller.wait(x)`, where `x` has to be replaced by the time to wait for, in seconds. Here is an example, where the agent goes forward during 3 seconds, then turns right during 0.5 second, then goes forward again during 2 seconds, and finally stops.

In [17]:
# Move forward, by setting each wheel speed at the same positive value
agent.left_motor = 1.
agent.right_motor = 1.

# Wait for 3 seconds, keeping moving forward at the same speed
controller.wait(3)

# Turn right by setting a positive speed to the left wheel and stoping the right one (null speed)
agent.left_motor = 0.5
agent.right_motor = 0.

# Keep turning for 0.5 second
controller.wait(0.5)

# Move forward again for 3 seconds
agent.left_motor = 1.
agent.right_motor = 1.
controller.wait(3)

# Stop the agent
agent.left_motor = 0.
agent.right_motor = 0.

A few important remarks regarding the code above:
- each line beginning with a `#` symbol corresponds to a comment and will therefore not be executed (it is just to explain verbally what the code does),
- when calling `controller.wait(.)`, the cell that is currently running will simply wait for the given time in seconds before executing the next line of code. this also means that you cannot execute anything else before the wait time is over.

Now, let's repeat the previous choreography 4 times. In the code below, the first line, `for i in range(4):`, means *repeat 4 times the indented code below* (actually it is a bit more complicated than this, but this out of the scope of the current session).  The indented code is the exact same as just before (only the comments have been removed). You can change the number `4` in the first line by any number `x`, to repeat it `x` times instead of 4.

In [18]:
for i in range(4):
    agent.left_motor = 1.
    agent.right_motor = 1.
    controller.wait(3)
    agent.left_motor = 1.
    agent.right_motor = 0.
    controller.wait(0.5)
    agent.left_motor = 1.
    agent.right_motor = 1.
    controller.wait(2)
    agent.left_motor = 0.
    agent.right_motor = 0.

**Q5:** By executing the code just above, you will observe that the agent actually only stops at the very end of the choreography, but not between each run of the loop. Copy-paste the code of the last cell above in the cell below and add one line of code so that the agent will stop for 1 second at each run of the loop.

**Q6:** Write the code allowing the agent to move roughly in a "8" shape. To do so you'll have to fine tune the waiting times by trial on error. Then make the agent repeat it 6 times.

### Sensors

As shown in [the quickstart tutorial](https://github.com/clement-moulin-frier/vivarium/blob/main/notebooks/quickstart_tutorial.ipynb), the agents are equipped with proximity sensors, called proximeters that indicate the presence of an entity, if any, in a particular direction. There are two sensor, one on each side of the agent, that detects entities in a hemi-field show in red in the interface.

Using the commands learned before, move the front of the robot close to an object in the scene and stop it. Alternatively, you can simply move the agent using drag&drop (see the [`web_interface_tutorial.md`](https://github.com/clement-moulin-frier/vivarium/blob/main/notebooks/web_interface_tutorial.md)).

Whenever a proximeter detects an obstacle, a red point is shown on the agent (if the `visible proximeters` checkbox is enabled). The opacity of the point depends on the activation of the sensor, so it can be hard to see if the entity is far away. Note that each proximeter detects only the closest object within its own field of view (indicate by the red lines). The current activation of the two front proximeters can be accessed with:

In [19]:
agent.sensors()

[0.5135992169380188, 0.657072901725769]

This command returns an `array`, i.e. a list of values. Here we see two values: the first one corresponds to the activity of the left proximeter, the second to the activity of the right one. Those values do NOT correspond to distances, but are instead activation values bounded between 0 and 1, where 0 indicates that no obstacle is perceived (i.e. the proximeter doesn't detect any object withing its field of view), and 1 indicates that an object is in contact with the proximeter. In other word, the higher the activation, the closer the object (inversely proportional to the object distance). 

Make sure that you are able to identify which sensor corresponds to which value. 

You can store the values returned by `agent.sensors()` in two distinct variables, which will facilitate using them for further computation. If you execute the following command:

In [20]:
left, right = agent.sensors()

Then the variables `left` and `right` will contain the activations of the left and right proximeters, respectively. You can check their values using the `print` instruction:

In [21]:
print(left)

0.5135992169380188


In [22]:
print(right)

0.657072901725769


**However**, the values in `left` and `right` will not be updated if the proximeter activations change over time. To experience it, move the robot in a different location in the scene where the proximeter activations should be different, and re-execute the `print` commands above. You will observe that the returned values are the same as before, whereas the proximeter activities should be different. In order to refresh the values in the `left` and `right` variables, you need to re-execute the command `left, right = agent.sensors()`:

In [23]:
left, right = agent.sensors()
print(left, right)

0.5135992169380188 0.657072901725769


In [24]:
agent.sensors()

[0.5135992169380188, 0.657072901725769]

Now you see that the values have been updated correctly.

**Q7:** To complete this session, write a behavior that connects sensors to motors, so that the robot goes forward at a speed that depends on the maximum activity of both proximity sensors. The closer the robot is from an obstacle, the slower it should go (but always going forward, without rotation). Remember that proximeter values have to be refreshed explicitly each time we need to observe them by calling `left, right = agent.sensors()` To do so, use the `for` loop construction we have seen, this time with a large number of iterations. Therefore, the code structure of this behavior will look like:

In [25]:
# Repeat 500 times:
for i in range(500):
    # Read the proximeter values and store them in the left and right variables
    left, right = agent.sensors()
    max_val = max(left, right)

    # Write your code below, which has to set the wheel speeds according to the max of left and right
    # Remember to keep your code indented, otherwise it will not be considered as part of the for loop
    # Also remember that the motor values should be between 0 and 1
    
    agent.left_motor = agent.right_motor = max_val
    
    # Keep the line below at the end of the loop. It waits for 100 milliseconds before starting the next iteration (to avoid overloading you computer)
    controller.wait(0.1)

The code above will take a while to be executed (approximately `500 * 0.1 = 50` seconds). During this time, you can't execute anything else in this notebook. To stop the execution before it terminates by itself, press the "stop-like" button in the top menu bar of this document. 

In the next practical session, we will see more fancy ways of defining reactive behaviours for the robot. And in the next ones, still more and more fancy stuffs.
You can now close the simulator session by executing:

In [19]:
controller.stop()
stop_server_and_interface()

vivarium/scripts/run_interface.py PID: 29994
Received signal 15, shutting down
vivarium/scripts/run_server.py PID: 29756
Server and Interface Stopped


Now that you finished session 1, you can now jump to the notebook of [section 2](session_2.ipynb).